#

# input layerの入力表現

## 流れ

1. パッチへ分割
   1. 画像をパッチに分割する
   2. flattenして、パッチの縦×横×チャンネルの長さのベクトルに変換する
2. パッチ埋め込み
   1. 1層の線形層で良いベクトルに変換する
   2. 良いベクトル:=損失が少ないベクトル
3. クラストークンを定義
   1. 画像全体の情報を保持する
   2. パッチ埋め込みのベクトルの大きさのベクトル
   3. 標準正規分布に従った値を設定
   4. クラストークンは学習のパラメータ
4. 位置埋め込み
   1. パッチ埋め込みだけではパッチの位置情報がないため、位置をクラストークンとパッチ埋め込みベクトルに埋め込む
   2. 初期値は標準正規分布に従う乱数を設定


## 数式表現

### 1. 入力画像からflattenまで

$H$: 画像の高さ（pixel）

$W$: 画像の幅（pixel）

$C$: 色情報（RGB）

$N_p$: パッチの個数 

$P$: パッチの縦横のサイズ（pixel）

とすると

入力画像を

> $\boldsymbol{x} \in \mathbb{R}^{H \times W \times C}$、


flattenした入力画像を

> $\boldsymbol{x_p} \in \mathbb{R}^{N_p\times(P^2 \cdot C)}$
と表せる。

flattenした各バッチのベクトルは

> $x_p^1, x_p^2, \dots, x_p^{N_p}$
> 
> と表す。


### 2. パッチ埋め込み

$D$ を 「$\boldsymbol{x_p}$ を埋め込んだベクトルの次元」とすると、$\boldsymbol{x_p}$を埋め込む線形層の重みは

> $E \in \mathbb{R}^{P^2 \cdot C} \times D$ 

と表せ、各バッチに$E$を適用すると

> $x_{p}^i E \in \mathbb{R}^D \quad (i = 1, 2, \dots, N_p)$ 

であり、

> $x_p E = [ x_p^1 E; x_p^2 E; \dots, x_p^{N_p} E ] \in \mathbb{R}^{N_p \times D}$

である。


### 3. クラストークン


$x_p E$ にクラストークンを付加する。$x_{class} \in \mathbb{R}^D$ を$x_p E$に追加して、

> $x_{p+t} E = [ x_{class}; x_p^1E;  x_p^2E; \dots, x_p^{N_p}E] \in \mathbb{R}^{(N_p + 1) \times D}$

である。

### 4. 位置埋め込み

トークン数を「クラストークン+パッチ数」とすると、トークン数は$N_p + 1$である。

位置埋め込みはD次元ベクトルがトークン数、すなわち

> $E_{pos} = [E x_{pos}^{class}; E x_{pos}^1; E x_{pos}^2; \dots; E x_{pos}^{N_p}] \in \mathbb{R}^{(N_p + 1) \times D}$

であるから、位置埋め込みしたEncoderへの入力$z_0$は

> $z_0 = x_{p+t}E + E_{pos} \in \mathbb{R}^{(N_p + 1) \times D} $

である。